<a href="https://colab.research.google.com/github/bryanrtboy/2DShapes_Drawing_Unity/blob/master/LoRAGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Start clean
import os, platform
os.chdir("/content")

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print("Python:", platform.python_version())

# 1) Remove conflicting wheels
!pip -q uninstall -y numpy opencv-python opencv-contrib-python opencv-python-headless >/dev/null 2>&1

# 2) Install versions that play nicely together
!pip -q install "numpy==1.26.4"
# This OpenCV build is known-good with NumPy 1.26.x on Colab
!pip -q install "opencv-python-headless==4.8.1.78"

# 3) Re-pin the rest of the stack (same stable set we used)
!pip -q uninstall -y diffusers huggingface_hub transformers accelerate bitsandbytes datasets peft >/dev/null 2>&1
!pip -q install "torch==2.3.1" "torchvision==0.18.1" --index-url https://download.pytorch.org/whl/cu121
!pip -q install "huggingface_hub==0.23.2" "diffusers==0.25.0" "transformers==4.41.1" "safetensors==0.4.3" "datasets==2.20.0" ftfy

# 4) Verify cv2 + numpy load cleanly
import numpy as np, cv2
print("NumPy:", np.__version__, "| OpenCV:", cv2.__version__)

# 5) Ensure sd-scripts present
if not os.path.exists("/content/sd-scripts/train_network.py"):
    !git clone https://github.com/kohya-ss/sd-scripts.git /content/sd-scripts
%cd /content/sd-scripts
!pip -q install -r requirements.txt

# 6) Warm tokenizer (avoid first-time stall)
from transformers import CLIPTokenizerFast
_ = CLIPTokenizerFast.from_pretrained("openai/clip-vit-large-patch14")
print("Tokenizer ready.")


Mounted at /content/drive
Python: 3.11.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.24 requires opencv-python-headless>=4.9.0.80, which is not installed.
dopamine-rl 4.1.2 requires opencv-python>=3.4.8.29, which is not installed.
albumentations 2.0.8 requires opencv-python-headless>=4.9.0.80, which is not installed.
pymc 5.25.1 requires rich>=13.7.1, but you have rich 13.7.0 which is incompatible.
gradio 5.41.0 requires huggingface-hub<1.0,>=0.33.5, but you have huggingface-hub 0.24.5 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.24 requires opencv-python-headless>=4.9.0.80, but you have ope

In [7]:
import os, json, pathlib, subprocess
os.chdir("/content")

BASE   = "/content/drive/MyDrive/lora_base_models/v1-5-pruned-emaonly.safetensors"
TRAIN  = "/content/drive/MyDrive/lora_data/bryan_paintings"      # images + .txt
OUTTMP = "/content/out"                                          # local (fast)
OUTDRV = "/content/drive/MyDrive/lora_outputs/bryan_painterly"   # Drive
NAME   = "BryanL_Painterly_LoRA"

print("Base exists:", os.path.isfile(BASE))
pathlib.Path(OUTTMP).mkdir(parents=True, exist_ok=True)
pathlib.Path(OUTDRV).mkdir(parents=True, exist_ok=True)

# Dataset config (512 for VRAM safety; you can bump to 640/768 after this works)
cfg = {
  "datasets": [{
    "resolution": 512,
    "batch_size": 1,
    "enable_bucket": True,
    "min_bucket_reso": 256,
    "max_bucket_reso": 1024,
    "subsets": [{
      "image_dir": TRAIN,
      "caption_extension": ".txt",
      "num_repeats": 1
    }]
  }]
}
with open("/content/ds.json","w") as f: json.dump(cfg, f, indent=2)
print(open("/content/ds.json").read())

# ---- 1-step MUST-WRITE smoke test (bf16, no cache_latents) ----
cmd1 = """
python -u /content/sd-scripts/train_network.py \
  --pretrained_model_name_or_path "/content/drive/MyDrive/lora_base_models/v1-5-pruned-emaonly.safetensors" \
  --dataset_config "/content/ds.json" \
  --output_dir "/content/out" \
  --output_name "BryanL_Painterly_LoRA" \
  --network_module networks.lora \
  --network_dim 16 --network_alpha 16 \
  --optimizer_type adamw \
  --learning_rate 1e-4 --text_encoder_lr 5e-5 \
  --max_train_steps 1 \
  --save_every_n_steps 1 \
  --mixed_precision bf16 \
  --shuffle_caption \
  --gradient_checkpointing \
  --max_data_loader_n_workers 0 \
  --save_model_as safetensors 2>&1 | tee /content/train_one_step.log
"""
print(cmd1)
subprocess.run(cmd1, shell=True, check=False)

print("\nLocal out listing after 1-step:")
!ls -lah /content/out || true
print("\nLast 40 lines of log:")
!tail -n 40 /content/train_one_step.log || true

# ---- Full run (safe) ----
cmd2 = """
python -u /content/sd-scripts/train_network.py \
  --pretrained_model_name_or_path "/content/drive/MyDrive/lora_base_models/v1-5-pruned-emaonly.safetensors" \
  --dataset_config "/content/ds.json" \
  --output_dir "/content/out" \
  --output_name "BryanL_Painterly_LoRA" \
  --network_module networks.lora \
  --network_dim 16 --network_alpha 16 \
  --optimizer_type adamw \
  --learning_rate 1e-4 --text_encoder_lr 5e-5 \
  --max_train_steps 4000 \
  --save_every_n_steps 500 \
  --mixed_precision bf16 \
  --cache_latents \
  --shuffle_caption \
  --gradient_checkpointing \
  --max_data_loader_n_workers 0 \
  --save_model_as safetensors 2>&1 | tee /content/train_full.log
"""
print("\nStarting FULL run…")
subprocess.run(cmd2, shell=True, check=False)

print("\nLocal out listing after full run:")
!ls -lah /content/out || true

print("\nCopying to Drive…")
!cp -v /content/out/*.safetensors "/content/drive/MyDrive/lora_outputs/bryan_painterly/" 2>/dev/null || true
!ls -lah "/content/drive/MyDrive/lora_outputs/bryan_painterly" || true


Base exists: True
{
  "datasets": [
    {
      "resolution": 512,
      "batch_size": 1,
      "enable_bucket": true,
      "min_bucket_reso": 256,
      "max_bucket_reso": 1024,
      "subsets": [
        {
          "image_dir": "/content/drive/MyDrive/lora_data/bryan_paintings",
          "caption_extension": ".txt",
          "num_repeats": 1
        }
      ]
    }
  ]
}

python -u /content/sd-scripts/train_network.py   --pretrained_model_name_or_path "/content/drive/MyDrive/lora_base_models/v1-5-pruned-emaonly.safetensors"   --dataset_config "/content/ds.json"   --output_dir "/content/out"   --output_name "BryanL_Painterly_LoRA"   --network_module networks.lora   --network_dim 16 --network_alpha 16   --optimizer_type adamw   --learning_rate 1e-4 --text_encoder_lr 5e-5   --max_train_steps 1   --save_every_n_steps 1   --mixed_precision bf16   --shuffle_caption   --gradient_checkpointing   --max_data_loader_n_workers 0   --save_model_as safetensors 2>&1 | tee /content/train_one_ste

In [8]:
!cp -v /content/out/*.safetensors "/content/drive/MyDrive/lora_outputs/bryan_painterly/"


'/content/out/BryanL_Painterly_LoRA.safetensors' -> '/content/drive/MyDrive/lora_outputs/bryan_painterly/BryanL_Painterly_LoRA.safetensors'
'/content/out/BryanL_Painterly_LoRA-step00000001.safetensors' -> '/content/drive/MyDrive/lora_outputs/bryan_painterly/BryanL_Painterly_LoRA-step00000001.safetensors'
'/content/out/BryanL_Painterly_LoRA-step00000500.safetensors' -> '/content/drive/MyDrive/lora_outputs/bryan_painterly/BryanL_Painterly_LoRA-step00000500.safetensors'
'/content/out/BryanL_Painterly_LoRA-step00001000.safetensors' -> '/content/drive/MyDrive/lora_outputs/bryan_painterly/BryanL_Painterly_LoRA-step00001000.safetensors'
'/content/out/BryanL_Painterly_LoRA-step00001500.safetensors' -> '/content/drive/MyDrive/lora_outputs/bryan_painterly/BryanL_Painterly_LoRA-step00001500.safetensors'
'/content/out/BryanL_Painterly_LoRA-step00002000.safetensors' -> '/content/drive/MyDrive/lora_outputs/bryan_painterly/BryanL_Painterly_LoRA-step00002000.safetensors'
'/content/out/BryanL_Painterly